In [70]:
f = open('데이터 전처리/차별 문장_전처리 완료.txt')
lines = f.readlines()

In [4]:
#명사 단위로 토큰화할 mecab import  
from konlpy.tag import Mecab
mecab = Mecab()
print(mecab.nouns(lines[1]))

['너', '정신병', '줄', '정상']


In [5]:
#명사 추출하여 리스트에 담는 작업
nouns_list = []
for i in lines :
    noun_s = mecab.nouns(i)
    for j in noun_s:
        nouns_list.append(j)
print(nouns_list[:10])

['너', '너', '정신병', '줄', '정상', '너', '뭘', '거짓말', '넌', '발정']


In [37]:
#명사 출현 횟수 구해서 데이터 프레임에 담기
from collections import Counter
count = Counter(nouns_list)

noun_count = []
nouns = []

lst = []
for n, c in count.most_common():
    if len(n) >= 2 :
        set = [c,n]
        lst.append(set)

In [38]:
print(lst[:50])

[[4304, '여자'], [2373, '남자'], [1397, '사람'], [1155, '한남'], [905, '장애'], [879, '짱깨'], [839, '나라'], [706, '페미'], [699, '우리'], [666, '생각'], [570, '수준'], [565, '얼굴'], [527, '한국'], [500, '결혼'], [470, '정신'], [461, '주제'], [455, '어디'], [451, '하나'], [446, '냄새'], [426, '조선족'], [425, '머리'], [417, '자기'], [406, '중국'], [383, '거지'], [374, '전라도'], [373, '게이'], [348, '때문'], [344, '흑인'], [340, '인간'], [340, '부모'], [335, '문제'], [304, '나이'], [298, '인생'], [297, '엄마'], [293, '돼지'], [278, '정도'], [276, '소리'], [267, '공부'], [264, '김치'], [258, '출신'], [251, '군대'], [248, '여성'], [247, '이상'], [243, '거기'], [237, '누가'], [234, '그게'], [234, '요즘'], [231, '텐데'], [231, '자식'], [228, '능력']]


In [8]:
#명사|횟수 csv 파일 저장 
import csv

with open('word_counting.csv', 'w',newline='') as f :
    write = csv.writer(f)
    write.writerows(lst)

In [42]:
#명사|횟수 데이터 프레임 형성 
import pandas as pd 

nouns_count = pd.DataFrame(lst, columns=['n_count','noun'])
nouns_count.head()

,n_count,noun
0,4304,여자
1,2373,남자
2,1397,사람
3,1155,한남
4,905,장애


In [11]:
text = ""
for i in lines:
    text += i

len(text)

986762

In [12]:
#10000글자씩 끊어서 리스트에 담기
sentences = []
n = 5000
for i in range(198):
    sentences.append(text[n-5000:n])
    n += 5000

In [29]:
len(sentences[197])

1762

In [14]:
#개체어 인식기 사용 준비 
import json
import urllib3
from tqdm.notebook import tqdm

In [35]:
#개체명 인식어 구현 모델 코드 기반 
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"    
accessKey = "6bb2897a-5f54-445d-9365-29fbfa60e2e0"
analysisCode = "ner"

word_dic = {}
word_count = {}
for i in tqdm(sentences) :     
    requestJson = {  
        "argument": {
            "text": i,
            "analysis_code": analysisCode
        }
    }
        
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    fine = json.loads(response.data)
    step1 = fine['return_object']['sentence']
   
    for i in step1:
        ne = i['NE']
        for item in ne:
            ne_text = item['text']
            ne_type = item['type']
            word_dic[ne_text] = ne_type
            word_count[ne_type] = word_count.get(ne_type, 0) + 1
   


  0%|          | 0/198 [00:00<?, ?it/s]

In [36]:
word_dic

{'정신병': 'TMM_DISEASE',
 '투포환': 'CV_SPORTS',
 '엄마': 'CV_RELATION',
 '남편': 'CV_RELATION',
 '한녀': 'QT_COUNT',
 '페미니즘': 'TR_SOCIAL_SCIENCE',
 '고기': 'CV_FOOD',
 '밥': 'CV_FOOD',
 '한숟갈': 'QT_COUNT',
 '한번': 'QT_COUNT',
 '한남': 'QT_COUNT',
 '경찰': 'OGG_POLITICS',
 '전교생': 'CV_POSITION',
 '10명': 'QT_MAN_COUNT',
 '여름': 'DT_SEASON',
 '다리털': 'AM_PART',
 '과학': 'FD_SCIENCE',
 '톡소플라즈마': 'MT_CHEMICAL',
 '뇌': 'TM_CELL_TISSUE',
 '강남역': 'LC_OTHERS',
 '살인': 'CV_LAW',
 '유가족': 'CV_RELATION',
 '한국': 'LCP_COUNTRY',
 '핸드폰': 'TMI_HW',
 '네일': 'DT_DURATION',
 '백퍼': 'TM_COLOR',
 '롯데리아': 'OGG_FOOD',
 '개': 'AM_MAMMALIA',
 '아프리카tv': 'AFW_DOCUMENT',
 '몸': 'AM_PART',
 'kbs': 'OGG_MEDIA',
 '이순신': 'PS_NAME',
 '00년대생': 'QT_AGE',
 '80년대생': 'QT_AGE',
 '1': 'QT_ORDER',
 '2': 'QT_ORDER',
 '3': 'QT_ORDER',
 '1/4': 'QT_PERCENTAGE',
 '백인': 'CV_TRIBE',
 '흑인': 'CV_TRIBE',
 '1+1': 'QT_OTHERS',
 '혜자': 'PS_NAME',
 '100%': 'QT_PERCENTAGE',
 '꽃뱀': 'AM_REPTILIA',
 '술': 'CV_DRINK',
 '한국인': 'CV_TRIBE',
 '100년 후': 'DT_DURATION',
 '1020': 'QT_L

In [44]:
#명사_태그_횟수 합치는 작업
df1 = pd.DataFrame(list(word_dic.items()),
                   columns=['noun', 'tag'])
df2 = pd.DataFrame(list(word_count.items()),
                   columns=['tag', 't_count'])
df3 = pd.merge(df1,df2,on='tag', how = 'outer')
real_df = df3.sort_values('t_count',ascending=False)
real_df.head()


,noun,tag,t_count
1450,비례대표,CV_POSITION,4287
1292,마루타,CV_POSITION,4287
1312,장인,CV_POSITION,4287
1311,종특,CV_POSITION,4287
1310,노예사슬자,CV_POSITION,4287


In [45]:
noun_tag_df = pd.merge(nouns_count,real_df,on='noun', how = 'outer')
noun_tag_df.head()

,n_count,noun,tag,t_count
0,4304.0,여자,NaN,NaN
1,2373.0,남자,CV_RELATION,2869.0
2,1397.0,사람,NaN,NaN
3,1155.0,한남,QT_COUNT,1499.0
4,905.0,장애,NaN,NaN


In [46]:
noun_tag_df = noun_tag_df.sort_values('t_count',ascending=False)
noun_tag_df.head()

,n_count,noun,tag,t_count
9971,NaN,게,CV_POSITION,4287.0
1574,9.0,이혼녀,CV_POSITION,4287.0
10006,NaN,여성손님,CV_POSITION,4287.0
10007,NaN,뱞신,CV_POSITION,4287.0
10008,NaN,무법자아,CV_POSITION,4287.0


In [54]:
noun_tag_df.to_csv('명사 및 태그 횟수 정보.csv')
real_df.to_csv('개체명인식결과.csv')

In [22]:
#문장과 태그 정보를 얻어서 데이터 프레임 생성
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"    
accessKey = "6bb2897a-5f54-445d-9365-29fbfa60e2e0"
analysisCode = "ner"

tag_set = []
for i in tqdm(sentences) :     
    requestJson = {  
        "argument": {
            "text": i,
            "analysis_code": analysisCode
        }
    }
        
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    fine = json.loads(response.data)
    step1 = fine['return_object']['sentence']
    for i in step1:
        sentence = i['text']
        ne = i['NE']
        for item in ne:
            ne_text = item['text']
            ne_type = item['type']
            sub_set = (ne_text,ne_type,sentence)
            tag_set.append(sub_set)

  0%|          | 0/198 [00:00<?, ?it/s]

In [58]:
import pandas as pd
tag_set_df = pd.DataFrame(tag_set, columns=['noun','tag','sentence'])
tag_set_df = tag_set_df.sort_values('tag',ascending=True)
tag_set_df.head()

NameError: name 'tag_set' is not defined

In [25]:
tag_set_df.to_csv('명사_개체어_문장.csv')

In [68]:
#태그 사용된 문장 비율 확인하기 
openApiURL = "http://aiopen.etri.re.kr:8000/WiseNLU_spoken"    
accessKey = "6bb2897a-5f54-445d-9365-29fbfa60e2e0"
analysisCode = "ner"

count = 0
for i in tqdm(sentences) :     
    requestJson = {  
        "argument": {
            "text": i,
            "analysis_code": analysisCode
        }
    }
        
    http = urllib3.PoolManager()
    response = http.request(
        "POST",
        openApiURL,
        headers={"Content-Type": "application/json; charset=UTF-8", "Authorization" :  accessKey},
        body=json.dumps(requestJson)
    )

    fine = json.loads(response.data)
    step1 = fine['return_object']['sentence']
    for i in step1:
        if i['NE'] == [] :
            count += 1

  0%|          | 0/198 [00:00<?, ?it/s]

In [74]:
100-((count/len(lines))*100)

54.46543994278956